In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np 
import imutils
import easyocr
from pathlib import Path

In [2]:
folder = Path("D:/coding/projectm5/data")

In [17]:
def cropped(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    bfilter = cv2.bilateralFilter(gray, 15, 20, 20)
    edged = cv2.Canny(bfilter, 30, 200)

    keypoint = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoint)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    mask = np.zeros(gray.shape, np.uint8)
    location = None

    for contour in contours:
        # print(i)
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            max_x = max(approx[:,:,1])
            min_x = min(approx[:,:,1])
            max_y = max(approx[:,:,0])
            min_y = min(approx[:,:,0])

            x_lenght = max_x-min_x
            y_lenght = max_y-min_y

            if x_lenght > 500 and y_lenght > 500:
                location = approx
                break
        
    if location is not None:
        new_image = cv2.drawContours(mask, [location], 0, 255, -1)
        new_image = cv2.bitwise_and(img, img, mask=mask)

        (x, y) = np.where(mask==255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        cropped_image = gray[x1:x2+1, y1:y2+1]
        return 1, cropped_image

    return 0, gray

In [4]:
def detectID(path):
    result = []
    img = cv2.imread(path)
    img = img[1600: , 100:500]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # # plt.imshow(cv2.cvtColor(gray, cv2.COLOR_BGR2RGB))

    # bfilter = cv2.bilateralFilter(gray, 15, 30, 30)
    # edged = cv2.Canny(bfilter, 30, 200)
    # # plt.imshow(cv2.cvtColor(edged, cv2.COLOR_BGR2RGB))

    # keypoint = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # contours = imutils.grab_contours(keypoint)
    # contours = sorted(contours, key=cv2.contourArea, reverse=True)[:20]
    
    # mask = np.zeros(gray.shape, np.uint8)
    # location = None

    # for contour in contours:
    #     # print(i)
    #     approx = cv2.approxPolyDP(contour, 10, True)
    #     if len(approx) == 4:
    #         location = approx
    #         new_image = cv2.drawContours(mask, [location], 0, 255, -1)
            # new_image = cv2.bitwise_and(img, img, mask=mask)

            # (x, y) = np.where(mask==255)
            # (x1, y1) = (np.min(x), np.min(y))
            # (x2, y2) = (np.max(x), np.max(y))
            # cropped_image = gray[x1:x2+1, y1:y2+1]

    reader = easyocr.Reader(['en'])
    result = reader.readtext(gray)

    return result

In [18]:
i = 0
for jpg_file_path in list(folder.rglob('*.jpg')):
    # print(type(jpg_file_path))
    # jpg_file = open(jpg_file_path, "r")
    result = detectID(str(jpg_file_path))
    print(result, end=" ")

    original_img = cv2.imread(str(jpg_file_path))
    check, cropped_img = cropped(original_img)

    if len(result) > 0:
        Id = result[0][1].rjust(3, '0')
        if check:
            cv2.imwrite(f'data_label\\{Id}.jpg', cropped_img)
        else:
            cv2.imwrite(f'data_label\\{Id}-incomplete.jpg', cropped_img)
    else:
        cv2.imwrite(f'data_label\\noID-{i}.jpg', original_img)
        i += 1
        print("path : ", jpg_file_path)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[143, 27], [163, 27], [163, 51], [143, 51]], '2', 0.875966959836178)] 

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[146, 24], [182, 24], [182, 54], [146, 54]], '15', 0.9999910648798075)] 

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


...

path :  D:\coding\projectm5\data\1-1\1-1-drawing_page-0004.jpg
[([[148, 22], [184, 22], [184, 50], [148, 50]], '27', 1.0)] 

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[148, 22], [186, 22], [186, 52], [148, 52]], '13', 0.9999983141264758)] 

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


KeyboardInterrupt: 